In [9]:
#!/usr/bin/env python3
"""
VERIFICATION SCRIPT: Confirm Perfect Alignment
==============================================

This script verifies that:
1. x_train features match RNN checkpoint expectations
2. x_train features match network nodes
3. All three are IDENTICALLY aligned

This gives you 100% confidence for fair benchmarking.
"""

import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
import os

print("=" * 80)
print("ALIGNMENT VERIFICATION: x_train vs RNN vs Network")
print("=" * 80)

# ============================================================================
# STEP 1: Prepare x_train (your current preprocessing)
# ============================================================================
print("\n[STEP 1] Preparing x_train with corrected preprocessing...")

DATA_PATH = os.path.expanduser('/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files')

# Load data
tf_expression = pd.read_csv(
    os.path.join(DATA_PATH, '/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/TF(full).tsv'),
    sep='\t',
    header=0,
    index_col=0
)

gene_expression = pd.read_csv(
    os.path.join(DATA_PATH, '/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/Geneexpression (full).tsv'),
    sep='\t',
    header=0,
    index_col=0
)

net = pd.read_csv(
    os.path.join(DATA_PATH, '/home/christianl/Zhang-Lab/Zhang Lab Data/Full data files/network(full).tsv'),
    sep='\t'
    )

# Get ALL network nodes (CORRECTED approach)
network_tfs = set(net['TF'].unique())
network_genes = set(net['Gene'].unique())
network_nodes = network_tfs | network_genes

# Filter to network nodes
usable_features = [tf for tf in tf_expression.columns if tf in network_nodes]
x = tf_expression[usable_features]
y = gene_expression

# Create train-test split
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=888
)

print(f"✓ x_train prepared")
print(f"  - Shape: {x_train.shape}")
print(f"  - Features: {len(usable_features)}")
print(f"  - Feature names: {x_train.columns.tolist()[:5]} ... {x_train.columns.tolist()[-5:]}")

# ============================================================================
# STEP 2: Load RNN checkpoint
# ============================================================================
print("\n[STEP 2] Loading RNN checkpoint...")

try:
    RNN_PATH = os.path.expanduser('~/Zhang-Lab/Zhang Lab Data/Saved models/RNN/uncentered_data_RNN/signaling_model.v1.pt')
    checkpoint = torch.load(RNN_PATH, map_location='cpu', weights_only=False)
    
    rnn_n_features = checkpoint['state_dict']['input_layer.weights'].shape[0]
    
    # Try to get feature names from checkpoint
    if 'usable_tfs' in checkpoint:
        rnn_features = checkpoint['usable_tfs']
        print(f"✓ RNN checkpoint loaded")
        print(f"  - Expected features: {rnn_n_features}")
        print(f"  - Feature list found in checkpoint: Yes")
    else:
        rnn_features = None
        print(f"✓ RNN checkpoint loaded")
        print(f"  - Expected features: {rnn_n_features}")
        print(f"  - Feature list found in checkpoint: No (will infer)")
        
except Exception as e:
    print(f"✗ Could not load RNN checkpoint: {e}")
    rnn_n_features = None
    rnn_features = None

# ============================================================================
# STEP 3: Get network nodes
# ============================================================================
print("\n[STEP 3] Analyzing network nodes...")

network_tfs_list = sorted(network_tfs)
network_genes_list = sorted(network_genes)
network_nodes_list = sorted(network_nodes)

# Count how many expression TFs are in each category
expr_tfs_in_network_tfs = [tf for tf in tf_expression.columns if tf in network_tfs]
expr_tfs_in_network_genes = [tf for tf in tf_expression.columns if tf in network_genes]
expr_tfs_in_network_nodes = [tf for tf in tf_expression.columns if tf in network_nodes]

print(f"✓ Network analysis complete")
print(f"  - Network TFs (regulators): {len(network_tfs)}")
print(f"  - Network genes (targets): {len(network_genes)}")
print(f"  - Total network nodes: {len(network_nodes)}")
print(f"\n  Expression TFs in network:")
print(f"  - As regulators only: {len(expr_tfs_in_network_tfs)}")
print(f"  - As targets only: {len([tf for tf in expr_tfs_in_network_genes if tf not in network_tfs])}")
print(f"  - In network (any role): {len(expr_tfs_in_network_nodes)}")

# ============================================================================
# STEP 4: Compare feature counts
# ============================================================================
print("\n" + "=" * 80)
print("FEATURE COUNT COMPARISON")
print("=" * 80)

print(f"\n  x_train features:    {len(usable_features)}")
print(f"  RNN expects:         {rnn_n_features if rnn_n_features else 'N/A'}")
print(f"  Network nodes:       {len(expr_tfs_in_network_nodes)}")

counts_match = (
    len(usable_features) == rnn_n_features == len(expr_tfs_in_network_nodes)
)

if counts_match:
    print(f"\n  ✅ COUNTS MATCH PERFECTLY!")
else:
    print(f"\n  ⚠️  COUNTS DO NOT MATCH")
    if rnn_n_features:
        print(f"     x_train vs RNN: {len(usable_features) - rnn_n_features:+d}")
    print(f"     x_train vs network: {len(usable_features) - len(expr_tfs_in_network_nodes):+d}")

# ============================================================================
# STEP 5: Compare feature lists (most critical test!)
# ============================================================================
print("\n" + "=" * 80)
print("FEATURE LIST COMPARISON (CRITICAL)")
print("=" * 80)

# Convert to sorted sets for comparison
x_train_features = set(x_train.columns.tolist())
network_features = set(expr_tfs_in_network_nodes)

print(f"\n[TEST 1] x_train vs Network nodes")
print(f"─" * 80)

# Features in x_train but NOT in network
in_xtrain_not_network = x_train_features - network_features
# Features in network but NOT in x_train
in_network_not_xtrain = network_features - x_train_features

print(f"  Features in x_train but NOT in network: {len(in_xtrain_not_network)}")
if in_xtrain_not_network:
    print(f"    → {sorted(in_xtrain_not_network)}")

print(f"  Features in network but NOT in x_train: {len(in_network_not_xtrain)}")
if in_network_not_xtrain:
    print(f"    → {sorted(in_network_not_xtrain)}")

if len(in_xtrain_not_network) == 0 and len(in_network_not_xtrain) == 0:
    print(f"\n  ✅ x_train and network features are IDENTICAL!")
else:
    print(f"\n  ⚠️  Mismatch detected!")

# ---

if rnn_features:
    print(f"\n[TEST 2] x_train vs RNN checkpoint")
    print(f"─" * 80)
    
    rnn_features_set = set(rnn_features)
    
    # Features in x_train but NOT in RNN
    in_xtrain_not_rnn = x_train_features - rnn_features_set
    # Features in RNN but NOT in x_train
    in_rnn_not_xtrain = rnn_features_set - x_train_features
    
    print(f"  Features in x_train but NOT in RNN: {len(in_xtrain_not_rnn)}")
    if in_xtrain_not_rnn:
        print(f"    → {sorted(in_xtrain_not_rnn)}")
    
    print(f"  Features in RNN but NOT in x_train: {len(in_rnn_not_xtrain)}")
    if in_rnn_not_xtrain:
        print(f"    → {sorted(in_rnn_not_xtrain)}")
    
    if len(in_xtrain_not_rnn) == 0 and len(in_rnn_not_xtrain) == 0:
        print(f"\n  ✅ x_train and RNN features are IDENTICAL!")
    else:
        print(f"\n  ⚠️  Mismatch detected!")

# ============================================================================
# STEP 6: Check feature ORDER (important for some models)
# ============================================================================
print(f"\n[TEST 3] Feature order check")
print(f"─" * 80)

x_train_ordered = x_train.columns.tolist()
network_ordered = sorted(expr_tfs_in_network_nodes)

if rnn_features:
    rnn_ordered = rnn_features
    
    # Check if order matches
    x_vs_rnn_order = (x_train_ordered == rnn_ordered)
    
    print(f"  x_train feature order matches RNN: {x_vs_rnn_order}")
    
    if not x_vs_rnn_order:
        print(f"\n  Note: Order mismatch is OK if features are the same.")
        print(f"  Models will still work correctly.")
else:
    print(f"  Cannot check order (RNN feature list not in checkpoint)")

# ============================================================================
# STEP 7: Check for 'TF' gene specifically
# ============================================================================
print(f"\n[TEST 4] 'TF' gene verification")
print(f"─" * 80)

tf_in_xtrain = 'TF' in x_train_features
tf_in_network = 'TF' in network_features
tf_in_rnn = 'TF' in rnn_features_set if rnn_features else None

print(f"  'TF' in x_train:   {tf_in_xtrain}")
print(f"  'TF' in network:   {tf_in_network}")
print(f"  'TF' in RNN:       {tf_in_rnn if tf_in_rnn is not None else 'N/A'}")

if tf_in_xtrain and tf_in_network:
    # Check how 'TF' appears in network
    tf_as_regulator = 'TF' in network_tfs
    tf_as_target = 'TF' in network_genes
    
    print(f"\n  'TF' gene in network as:")
    print(f"    - Regulator (TF column): {tf_as_regulator}")
    print(f"    - Target (Gene column): {tf_as_target}")
    
    if tf_as_target:
        tf_regulators = net[net['Gene'] == 'TF']
        print(f"    - Number of regulators: {len(tf_regulators)}")
        print(f"    - Sample regulators: {tf_regulators['TF'].head(5).tolist()}")

# ============================================================================
# STEP 8: Summary table
# ============================================================================
print("\n" + "=" * 80)
print("ALIGNMENT SUMMARY TABLE")
print("=" * 80)

print(f"""
┌─────────────────────────────┬────────────┬──────────┬──────────┐
│ Component                   │ x_train    │ RNN      │ Network  │
├─────────────────────────────┼────────────┼──────────┼──────────┤
│ Feature count               │ {len(usable_features):<10} │ {rnn_n_features if rnn_n_features else 'N/A':<8} │ {len(expr_tfs_in_network_nodes):<8} │
│ Includes 'TF' gene          │ {str(tf_in_xtrain):<10} │ {str(tf_in_rnn) if tf_in_rnn is not None else 'N/A':<8} │ {str(tf_in_network):<8} │
│ Features identical          │ ─          │ {('Yes' if rnn_features and x_train_features == rnn_features_set else 'No' if rnn_features else 'N/A'):<8} │ {('Yes' if x_train_features == network_features else 'No'):<8} │
└─────────────────────────────┴────────────┴──────────┴──────────┘
""")

# ============================================================================
# FINAL VERDICT
# ============================================================================
print("=" * 80)
print("FINAL VERDICT")
print("=" * 80)

all_tests_passed = True
test_results = []

# Test 1: Feature count
if len(usable_features) == len(expr_tfs_in_network_nodes):
    test_results.append("✅ Feature count: x_train matches network")
else:
    test_results.append("⚠️  Feature count: MISMATCH")
    all_tests_passed = False

# Test 2: Feature list
if x_train_features == network_features:
    test_results.append("✅ Feature list: x_train matches network")
else:
    test_results.append("⚠️  Feature list: MISMATCH")
    all_tests_passed = False

# Test 3: RNN match
if rnn_n_features:
    if len(usable_features) == rnn_n_features:
        test_results.append("✅ Feature count: x_train matches RNN")
    else:
        test_results.append("⚠️  Feature count: x_train vs RNN MISMATCH")
        all_tests_passed = False
    
    if rnn_features:
        if x_train_features == rnn_features_set:
            test_results.append("✅ Feature list: x_train matches RNN")
        else:
            test_results.append("⚠️  Feature list: x_train vs RNN MISMATCH")
            all_tests_passed = False

# Test 4: 'TF' gene
if tf_in_xtrain and tf_in_network:
    test_results.append("✅ 'TF' gene: Included in both x_train and network")
elif not tf_in_xtrain and not tf_in_network:
    test_results.append("⚠️  'TF' gene: Missing from both (unexpected)")
    all_tests_passed = False
else:
    test_results.append("⚠️  'TF' gene: Inconsistent presence")
    all_tests_passed = False

print("\nTest Results:")
for result in test_results:
    print(f"  {result}")

print("\n" + "─" * 80)

if all_tests_passed:
    print("""
🎉 ALL TESTS PASSED! 🎉

Your x_train features are PERFECTLY aligned with:
✓ Network topology
✓ RNN checkpoint expectations
✓ Includes 'TF' gene correctly

You can proceed with confidence:
1. Train MLR on x_train
2. Train XGBRF on x_train  
3. Use existing RNN checkpoint
4. Fair benchmarking guaranteed!

All models will see IDENTICAL feature sets.
""")
else:
    print("""
⚠️  ALIGNMENT ISSUES DETECTED ⚠️

Please review the mismatches above and correct your preprocessing.

Common fixes:
1. Ensure you're using: network_nodes = network_tfs | network_genes
2. Check random seed matches (888)
3. Verify index_col=0 in data loading
4. Confirm network.tsv file is correct version
""")

print("=" * 80)
print("VERIFICATION COMPLETE")
print("=" * 80)

ALIGNMENT VERIFICATION: x_train vs RNN vs Network

[STEP 1] Preparing x_train with corrected preprocessing...
✓ x_train prepared
  - Shape: (12748, 1197)
  - Features: 1197
  - Feature names: ['ADNP', 'ADNP2', 'AEBP1', 'AEBP2', 'AHR'] ... ['ZSCAN4', 'ZSCAN5A', 'ZSCAN9', 'ZXDA', 'ZXDC']

[STEP 2] Loading RNN checkpoint...
✓ RNN checkpoint loaded
  - Expected features: 1197
  - Feature list found in checkpoint: No (will infer)

[STEP 3] Analyzing network nodes...
✓ Network analysis complete
  - Network TFs (regulators): 1200
  - Network genes (targets): 16292
  - Total network nodes: 16371

  Expression TFs in network:
  - As regulators only: 1196
  - As targets only: 1
  - In network (any role): 1197

FEATURE COUNT COMPARISON

  x_train features:    1197
  RNN expects:         1197
  Network nodes:       1197

  ✅ COUNTS MATCH PERFECTLY!

FEATURE LIST COMPARISON (CRITICAL)

[TEST 1] x_train vs Network nodes
────────────────────────────────────────────────────────────────────────────────

In [7]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate_datahandling/Remote boilerplate/model_boilerplate_remote.py'

KeyboardInterrupt: 

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# out-of-the-box r2 score LinearRegression:
# train score: 0.90870454039286
# test score:  0.7934193600256717 
# difference of 0.12 between train and test is understandable given the high-dimensionality and noise in gene expression data

# after applying the centering boilerplate (only difference between MLR v1 and v2 is column-wise centering)
# train score: 0.90870454039286
# test score:  0.7934193600256717 

# after working with uncentered, paired data from the network.tsv file (only TFs in that can be included)
# train score:  0.908486689146908
# test score:  0.7993502011028869


from sklearn.linear_model import LinearRegression

reg_test = LinearRegression(
                    n_jobs=-1,
).fit(x_train, y_train)

In [4]:
print('Score: ', reg_test.score(x_train, y_train))
print('Score: ', reg_test.score(x_test, y_test))

Score:  0.908486689146908
Score:  0.7993502011028869


In [5]:
# conducting a k-fold validation to check for overfitting
# splitting training set into 10-folds
# train sets: ~10038 targets
# test sets: ~1116 targets

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cnt = 0
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=888) # changed from 42 to 888 to match training seed for RNN 13/01/26
for train_index, test_index in kf.split(x_train, y_train):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, \
    Test set:{len(test_index)}')
    cnt += 1

Fold:0, Train set: 10038,     Test set:1116
Fold:1, Train set: 10038,     Test set:1116
Fold:2, Train set: 10038,     Test set:1116
Fold:3, Train set: 10038,     Test set:1116
Fold:4, Train set: 10039,     Test set:1115
Fold:5, Train set: 10039,     Test set:1115
Fold:6, Train set: 10039,     Test set:1115
Fold:7, Train set: 10039,     Test set:1115
Fold:8, Train set: 10039,     Test set:1115
Fold:9, Train set: 10039,     Test set:1115


In [6]:
# running the 10-fold validation 
#----- MLR model cross validation ------
#Scores: [0.78811923 0.79602325 0.7830914  0.78294147 0.78665107 0.7712694
# 0.77717178 0.7840341  0.79817987 0.76680642]
#Mean: 0.7834287993694773 -> similar to the holdout training run
#StandardDeviation: 0.009341412328795035 -> very consistent between folds

# unlikely that MLR is overfitting based on these results -> good sanity check for reference

# redid same analysis with column-wise centered data and got the exact same results 
#----- MLR model cross validation ------
#Scores: [0.78811923 0.79602325 0.7830914  0.78294147 0.78665107 0.7712694
# 0.77717178 0.7840341  0.79817987 0.76680642]
#Mean: 0.7834287993694773
#StandardDeviation: 0.00934141232879502

# redid same analysis once again, with uncentered data like the first run, but set random seed to 888
#----- MLR model cross validation ------
#Scores: [0.79322189 0.79384094 0.80615516 0.77715731 0.78278073 0.78325776
# 0.79921787 0.77486414 0.75888273 0.7807757 ]
#Mean: 0.7850154223167911
#StandardDeviation: 0.01291359135998445


# same as before but used index_col=0 and removed 'TF' which was a junk filler column 15/01/26
#----- MLR model cross validation ------
#Scores: [0.79306553 0.79355143 0.80595273 0.77689887 0.78254468 0.78294832
# 0.79897946 0.77465899 0.75861371 0.78061954]
#Mean: 0.7847833249078063
#StandardDeviation: 0.012924317642343348


def cross_validation(reg_model, training_set, training_target, cv):
    scores = cross_val_score(
      reg_model, training_set,
      training_target,
      scoring="r2", cv=cv)
    r2_scores = scores
    print("Scores:", r2_scores)
    print("Mean:", r2_scores.mean())
    print("StandardDeviation:", r2_scores.std())

print("----- MLR model cross validation ------")
lin_reg = LinearRegression()
cross_validation(lin_reg, x_train, y_train, kf)
print("")

----- MLR model cross validation ------
Scores: [0.79306553 0.79355143 0.80595273 0.77689887 0.78254468 0.78294832
 0.79897946 0.77465899 0.75861371 0.78061954]
Mean: 0.7847833249078063
StandardDeviation: 0.012924317642343348



In [6]:
import joblib
import json

# saving model + metadata

metadata = {
    'sklearn_version': '1.7.2',
    'model_type': 'LinearRegression',
    'n_features': reg_test.n_features_in_,
    'coef_shape': reg_test.coef_.shape,
    'intercept_shape': reg_test.intercept_.shape,
}

joblib.dump(reg_test, "/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/MLR/MLR_v3/MLR_model_v4(uncentered[FINAL]).joblib")
with open("/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/MLR/MLR_v3/model_metadata_uncentered[FINAL].json", "w") as f:
    json.dump(metadata, f)